## 15. Transform Orders Data - String to JSON

In [0]:
%sql
SELECT * FROM gizmobox.bronze.v_orders

## 1. Preprocess the JSON string to fix the data quality issues

In [0]:
%sql
SELECT 
  value,
  regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
 FROM gizmobox.bronze.v_orders

## 2. Extract array elements

In [0]:
%sql
SELECT 
 schema_of_json(regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"')) AS schema,
 regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value   
 FROM gizmobox.bronze.v_orders
 LIMIT 1

In [0]:
%sql
SELECT 
  from_json(regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"'),
  'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value
 FROM gizmobox.bronze.v_orders
 LIMIT 1

## 3. Save Silver

In [0]:
%sql
CREATE TABLE gizmobox.silver.orders_json
AS
SELECT 
  from_json(regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"'),
  'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value
 FROM gizmobox.bronze.v_orders;

In [0]:
%sql
SELECT * FROM gizmobox.silver.orders_json;